In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')/255.0
        self.test_x_f1 = np.load('../testImage_canny.npy')/255.0
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.x_f1: batch_x_f1_test,
                                                                   self.x_f2: batch_x_f2_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 21.30, total 50 epochs ptime: 1084.07
#  [*] Training finished!
#  [*] Best Epoch:  15 , Accuracy:  0.8800000548362732
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-15
#  [*] Finished testing Best Epoch: 15 , accuracy:  0.8799999952316284 

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32

Epoch: [ 2] [  36/  60] time: 36.3558, loss: 0.03510561
Epoch: [ 2] [  37/  60] time: 36.6930, loss: 0.04571804
Epoch: [ 2] [  38/  60] time: 37.0338, loss: 0.07231341
Epoch: [ 2] [  39/  60] time: 37.3724, loss: 0.05120147
Epoch: [ 2] [  40/  60] time: 37.7095, loss: 0.01408680
Epoch: [ 2] [  41/  60] time: 38.0478, loss: 0.06428532
Epoch: [ 2] [  42/  60] time: 38.3877, loss: 0.04825547
Epoch: [ 2] [  43/  60] time: 38.7262, loss: 0.00989006
Epoch: [ 2] [  44/  60] time: 39.0651, loss: 0.06668387
Epoch: [ 2] [  45/  60] time: 39.4037, loss: 0.02307392
Epoch: [ 2] [  46/  60] time: 39.7410, loss: 0.03382893
Epoch: [ 2] [  47/  60] time: 40.0793, loss: 0.00754506
Epoch: [ 2] [  48/  60] time: 40.4154, loss: 0.02701989
Epoch: [ 2] [  49/  60] time: 40.7540, loss: 0.04173492
Epoch: [ 2] [  50/  60] time: 41.0943, loss: 0.05884765
Epoch: [ 2] [  51/  60] time: 41.4328, loss: 0.02232390
Epoch: [ 2] [  52/  60] time: 41.7712, loss: 0.02260304
Epoch: [ 2] [  53/  60] time: 42.1111, loss: 0.0

Epoch: [ 5] [   0/  60] time: 88.9918, loss: 0.02435905
Epoch: [ 5] [   1/  60] time: 89.3286, loss: 0.02112989
Epoch: [ 5] [   2/  60] time: 89.6681, loss: 0.01212202
Epoch: [ 5] [   3/  60] time: 90.0090, loss: 0.01324551
Epoch: [ 5] [   4/  60] time: 90.3472, loss: 0.01237673
Epoch: [ 5] [   5/  60] time: 90.6852, loss: 0.00924705
Epoch: [ 5] [   6/  60] time: 91.0259, loss: 0.04412877
Epoch: [ 5] [   7/  60] time: 91.3639, loss: 0.03903977
Epoch: [ 5] [   8/  60] time: 91.7013, loss: 0.02075708
Epoch: [ 5] [   9/  60] time: 92.0416, loss: 0.01245862
Epoch: [ 5] [  10/  60] time: 92.3792, loss: 0.03100399
Epoch: [ 5] [  11/  60] time: 92.7165, loss: 0.02575957
Epoch: [ 5] [  12/  60] time: 93.0571, loss: 0.00286668
Epoch: [ 5] [  13/  60] time: 93.3987, loss: 0.02226756
Epoch: [ 5] [  14/  60] time: 93.7363, loss: 0.04784891
Epoch: [ 5] [  15/  60] time: 94.0764, loss: 0.01027613
Epoch: [ 5] [  16/  60] time: 94.4155, loss: 0.01310301
Epoch: [ 5] [  17/  60] time: 94.7526, loss: 0.0

Epoch: [ 7] [  22/  60] time: 139.6554, loss: 0.01983922
Epoch: [ 7] [  23/  60] time: 139.9943, loss: 0.00496201
Epoch: [ 7] [  24/  60] time: 140.3320, loss: 0.02935800
Epoch: [ 7] [  25/  60] time: 140.6703, loss: 0.03632838
Epoch: [ 7] [  26/  60] time: 141.0121, loss: 0.03750445
Epoch: [ 7] [  27/  60] time: 141.3512, loss: 0.02471052
Epoch: [ 7] [  28/  60] time: 141.6913, loss: 0.01274417
Epoch: [ 7] [  29/  60] time: 142.0351, loss: 0.03599373
Epoch: [ 7] [  30/  60] time: 142.3771, loss: 0.04587183
Epoch: [ 7] [  31/  60] time: 142.7163, loss: 0.06676758
Epoch: [ 7] [  32/  60] time: 143.0589, loss: 0.01246332
Epoch: [ 7] [  33/  60] time: 143.3975, loss: 0.03045180
Epoch: [ 7] [  34/  60] time: 143.7374, loss: 0.00813578
Epoch: [ 7] [  35/  60] time: 144.0793, loss: 0.00129019
Epoch: [ 7] [  36/  60] time: 144.4173, loss: 0.00525835
Epoch: [ 7] [  37/  60] time: 144.7562, loss: 0.07504163
Epoch: [ 7] [  38/  60] time: 145.0964, loss: 0.01707852
Epoch: [ 7] [  39/  60] time: 1

Epoch: [ 9] [  44/  60] time: 190.4847, loss: 0.01186045
Epoch: [ 9] [  45/  60] time: 190.8227, loss: 0.00783551
Epoch: [ 9] [  46/  60] time: 191.1632, loss: 0.00155735
Epoch: [ 9] [  47/  60] time: 191.5023, loss: 0.00476051
Epoch: [ 9] [  48/  60] time: 191.8421, loss: 0.01150380
Epoch: [ 9] [  49/  60] time: 192.1820, loss: 0.00845349
Epoch: [ 9] [  50/  60] time: 192.5215, loss: 0.02084713
Epoch: [ 9] [  51/  60] time: 192.8618, loss: 0.00445549
Epoch: [ 9] [  52/  60] time: 193.2033, loss: 0.00393829
Epoch: [ 9] [  53/  60] time: 193.5408, loss: 0.00083936
Epoch: [ 9] [  54/  60] time: 193.8808, loss: 0.01137753
Epoch: [ 9] [  55/  60] time: 194.2190, loss: 0.00054633
Epoch: [ 9] [  56/  60] time: 194.5574, loss: 0.00597352
Epoch: [ 9] [  57/  60] time: 194.8951, loss: 0.00653877
Epoch: [ 9] [  58/  60] time: 195.2324, loss: 0.00171478
Epoch: [ 9] [  59/  60] time: 195.5725, loss: 0.00660163
[9/50] - ptime: 21.2556 loss: 0.01350358 acc: 0.74000 lr: 0.00100000
Epoch: [10] [   0/ 

...

Epoch: [40] [  25/  60] time: 855.4450, loss: 0.00000077
Epoch: [40] [  26/  60] time: 855.7832, loss: 0.00000188
Epoch: [40] [  27/  60] time: 856.1254, loss: 0.00002174
Epoch: [40] [  28/  60] time: 856.4652, loss: 0.00000573
Epoch: [40] [  29/  60] time: 856.8026, loss: 0.00001261
Epoch: [40] [  30/  60] time: 857.1442, loss: 0.00000953
Epoch: [40] [  31/  60] time: 857.4895, loss: 0.00000827
Epoch: [40] [  32/  60] time: 857.8312, loss: 0.00012637
Epoch: [40] [  33/  60] time: 858.1706, loss: 0.00000004
Epoch: [40] [  34/  60] time: 858.5103, loss: 0.00002395
Epoch: [40] [  35/  60] time: 858.8500, loss: 0.00004555
Epoch: [40] [  36/  60] time: 859.1890, loss: 0.00000315
Epoch: [40] [  37/  60] time: 859.5380, loss: 0.00000013
Epoch: [40] [  38/  60] time: 859.8782, loss: 0.00000446
Epoch: [40] [  39/  60] time: 860.2201, loss: 0.00000070
Epoch: [40] [  40/  60] time: 860.5615, loss: 0.00000216
Epoch: [40] [  41/  60] time: 860.9013, loss: 0.00002435
Epoch: [40] [  42/  60] time: 8

Epoch: [42] [  47/  60] time: 906.2174, loss: 0.00003650
Epoch: [42] [  48/  60] time: 906.5572, loss: 0.00000032
Epoch: [42] [  49/  60] time: 906.8948, loss: 0.00000408
Epoch: [42] [  50/  60] time: 907.2328, loss: 0.00182026
Epoch: [42] [  51/  60] time: 907.5707, loss: 0.00000841
Epoch: [42] [  52/  60] time: 907.9085, loss: 0.00000192
Epoch: [42] [  53/  60] time: 908.2471, loss: 0.00000130
Epoch: [42] [  54/  60] time: 908.5850, loss: 0.00000141
Epoch: [42] [  55/  60] time: 908.9235, loss: 0.00013341
Epoch: [42] [  56/  60] time: 909.2625, loss: 0.00001447
Epoch: [42] [  57/  60] time: 909.6008, loss: 0.00000061
Epoch: [42] [  58/  60] time: 909.9393, loss: 0.00043781
Epoch: [42] [  59/  60] time: 910.2776, loss: 0.00000474
[42/50] - ptime: 21.2577 loss: 0.00005344 acc: 0.82000 lr: 0.00065610
Epoch: [43] [   0/  60] time: 911.8967, loss: 0.00000068
Epoch: [43] [   1/  60] time: 912.2334, loss: 0.00004261
Epoch: [43] [   2/  60] time: 912.5709, loss: 0.00000125
Epoch: [43] [   3/

Epoch: [45] [   8/  60] time: 957.8941, loss: 0.00015630
Epoch: [45] [   9/  60] time: 958.2328, loss: 0.00606237
Epoch: [45] [  10/  60] time: 958.5729, loss: 0.00227638
Epoch: [45] [  11/  60] time: 958.9130, loss: 0.00170733
Epoch: [45] [  12/  60] time: 959.2519, loss: 0.00467035
Epoch: [45] [  13/  60] time: 959.5909, loss: 0.04243997
Epoch: [45] [  14/  60] time: 959.9291, loss: 0.00103813
Epoch: [45] [  15/  60] time: 960.2690, loss: 0.00457681
Epoch: [45] [  16/  60] time: 960.6079, loss: 0.00127908
Epoch: [45] [  17/  60] time: 960.9452, loss: 0.03992480
Epoch: [45] [  18/  60] time: 961.2842, loss: 0.01462318
Epoch: [45] [  19/  60] time: 961.6227, loss: 0.00317865
Epoch: [45] [  20/  60] time: 961.9641, loss: 0.00664902
Epoch: [45] [  21/  60] time: 962.3027, loss: 0.07843532
Epoch: [45] [  22/  60] time: 962.6436, loss: 0.00212482
Epoch: [45] [  23/  60] time: 962.9825, loss: 0.00081192
Epoch: [45] [  24/  60] time: 963.3209, loss: 0.00033551
Epoch: [45] [  25/  60] time: 9

Epoch: [47] [  29/  60] time: 1008.3180, loss: 0.00006301
Epoch: [47] [  30/  60] time: 1008.6590, loss: 0.00023179
Epoch: [47] [  31/  60] time: 1008.9985, loss: 0.00060921
Epoch: [47] [  32/  60] time: 1009.3376, loss: 0.00004240
Epoch: [47] [  33/  60] time: 1009.6763, loss: 0.00018765
Epoch: [47] [  34/  60] time: 1010.0170, loss: 0.00025598
Epoch: [47] [  35/  60] time: 1010.3566, loss: 0.00027316
Epoch: [47] [  36/  60] time: 1010.6945, loss: 0.00008084
Epoch: [47] [  37/  60] time: 1011.0348, loss: 0.00103945
Epoch: [47] [  38/  60] time: 1011.3726, loss: 0.00009853
Epoch: [47] [  39/  60] time: 1011.7105, loss: 0.00009985
Epoch: [47] [  40/  60] time: 1012.0509, loss: 0.00017232
Epoch: [47] [  41/  60] time: 1012.3888, loss: 0.00007874
Epoch: [47] [  42/  60] time: 1012.7263, loss: 0.00015231
Epoch: [47] [  43/  60] time: 1013.0648, loss: 0.00080256
Epoch: [47] [  44/  60] time: 1013.4030, loss: 0.00027839
Epoch: [47] [  45/  60] time: 1013.7422, loss: 0.00013251
Epoch: [47] [ 

Epoch: [49] [  48/  60] time: 1058.1059, loss: 0.00030818
Epoch: [49] [  49/  60] time: 1058.4446, loss: 0.00011981
Epoch: [49] [  50/  60] time: 1058.7833, loss: 0.00002243
Epoch: [49] [  51/  60] time: 1059.1225, loss: 0.00091375
Epoch: [49] [  52/  60] time: 1059.4607, loss: 0.00002068
Epoch: [49] [  53/  60] time: 1059.7982, loss: 0.00011318
Epoch: [49] [  54/  60] time: 1060.1377, loss: 0.00004901
Epoch: [49] [  55/  60] time: 1060.4756, loss: 0.00006128
Epoch: [49] [  56/  60] time: 1060.8135, loss: 0.00128197
Epoch: [49] [  57/  60] time: 1061.1541, loss: 0.00005961
Epoch: [49] [  58/  60] time: 1061.4931, loss: 0.00203598
Epoch: [49] [  59/  60] time: 1061.8312, loss: 0.00004208
[49/50] - ptime: 21.2795 loss: 0.00034734 acc: 0.82000 lr: 0.00065610
Epoch: [50] [   0/  60] time: 1063.4525, loss: 0.00012442
Epoch: [50] [   1/  60] time: 1063.7901, loss: 0.00006238
Epoch: [50] [   2/  60] time: 1064.1299, loss: 0.00001530
Epoch: [50] [   3/  60] time: 1064.4683, loss: 0.00007141
Ep

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放